In [10]:
import pandas as pd

In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os

import requests

from dotenv import load_dotenv

import time

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Authentication - use of client cred
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret,
)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [16]:
import deezer

client = deezer.Client()


def get_song_bpm(track, artist):
    try:
        result = client.search(track=track, artist=artist)

        if result:
            print(f"getting bpm for {track} by {artist}")
            bpm = result[0].bpm
            if bpm:
                return bpm
            else:
                print("BPM not found")
                return None
        else:
            print("Song not found")
            return None

    # GPT-4 was used to generate error handling code here:
    except requests.exceptions.HTTPError as http_err:
        if http_err.response.status_code == 403:
            print(
                "403 Error: Access Forbidden. Please check your API key or authentication."
            )
        else:
            print(f"HTTP error occurred: {http_err}")  # For other HTTP errors
    except Exception as err:
        print(f"An error occurred: {err}")

In [17]:
# Client credentials method does not require authorization
def playlist_bpm(playlist_id, sp):
    playlist = sp.playlist_tracks(playlist_id)
    tracks = []
    artists = []
    bpm_list = []
    for item in playlist["items"]:
        artist = item["track"]["artists"][0]["name"]
        track = item["track"]["name"]

        tracks.append(item["track"]["name"])
        artists.append(item["track"]["artists"][0]["name"])

        bpm = get_song_bpm(track, artist)

        bpm_list.append(bpm)
        

    data = {
        "tracks": tracks,
        "artists": artists,
        "bpm": bpm_list
    }

    df = pd.DataFrame.from_dict(data)

    return df

In [18]:
playlist_id = "5oAd7leEjwjyXVSIZjov7O" #ID for IOT test playlist
df = playlist_bpm(playlist_id, sp)

getting bpm for Superfly by Curtis Mayfield
getting bpm for One Nation Under a Groove by Funkadelic
getting bpm for Flash Light by Parliament
BPM not found
getting bpm for Higher Ground by Stevie Wonder
getting bpm for Dancing in the Moonlight by King Harvest
getting bpm for Papa Was A Rollin' Stone - Single Version by The Temptations
getting bpm for Super Freak by Rick James
getting bpm for Play That Funky Music by Wild Cherry
getting bpm for Linus And Lucy by Vince Guaraldi Trio
getting bpm for Birdland by Weather Report
getting bpm for Mood Indigo by Nina Simone
BPM not found
getting bpm for Mood Indigo by Frank Sinatra
getting bpm for Strange Fruit by Billie Holiday
getting bpm for 'Round Midnight by Thelonious Monk
BPM not found
getting bpm for Take the "A" Train by Duke Ellington
getting bpm for Take Five by The Dave Brubeck Quartet
getting bpm for The Girl From Ipanema by Stan Getz
getting bpm for Fly Me To The Moon by Frank Sinatra
getting bpm for Summertime by The Sundays
gett

In [19]:
df.to_csv("bpm_dataset.csv")